In [29]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub;
from pathlib import Path
import pandas as pd;
import glob
from sklearn.preprocessing import MultiLabelBinarizer
import multiprocessing
import numpy as np
import json as json

class cfg:
    img_size_dest = (224,224)
    train_batchsize = 100
    test_batchsize = 100
    val_batchsize = 256
    num_epochs = 10
    lr=0.1 #10e-3
    weights = {
        "Agriculture":                                               67256 ,
        "Airports": 0,
        "Annual crops associated with permanent crops" : 0,
        "Bare rock": 0,
        "Beaches, dunes, sands":0,
        "Burnt areas":0,
        "Coastal lagoons":0,
        "Construction sites": 0,
        "Continuous urban fabric": 0,
        "Discontinuous urban fabric":0,
        "Estuaries":0,
        "Fruit trees and berry plantations":0,
        "Green urban areas":0,
        "Land principally occupied by agriculture, with significant areas of natural vegetation":0,
        "Mineral extraction sites":0,
        "Inland marshes":0,
        "Agro-forestry areas":					  15790 ,
        "Arable land":						 100394 ,
        "Beaches,dunes,sands":					   1193 ,
        "Broad-leaved forest":					  73407 ,
        "Coastal wetlands":						   1033 ,
        "Complex cultivation patterns":				  53530 ,
        "Coniferous forest":					  86569 ,
        "Dump sites":0,
        "Industrial or commercial units":				   6182 ,
        "Inland waters":						  35349 ,
        "Inland wetlands":						  11620 ,
        "Intertidal flats":0,
        "Marine waters":						  39110 ,
        "Mixed forest":						  91926 ,
        "Moors and heathland":0,
        "Natural grassland":0,
        "Moors,heathlandandsclerophyllousvegetation":		   8434 ,
        "Naturalgrasslandandsparselyvegetatedareas":		   6663 ,
        "Non-irrigated arable land":0,
        "Olive groves":0,
        "Peatbogs":0,
        "Pastures":						  50977 ,
        "Permanent crops":				  15862 ,
        "Port areas":0,
        "Rice fields":0,
        "Road and rail networks and associated land":0,
        "Permanently irrigated land":0,
        "Salt marshes":0,
        "Salines":0,
        "Transitional woodland/shrub":				  77589 ,
        "Sclerophyllous vegetation":0,
        "Sea and ocean":0,
        "Urbanfabric":					  38779,
        "Sport and leisure facilities":0,
        "Sparsely vegetated areas":0,
        "Vineyards":0,
        "Water bodies":0,
        "Water courses":0
}
#    steps_per_epoch = 256
    
    

def get_data(path_images):
    classes_arr = []
    files = glob.glob(str(path_images) + "/*/" + '*.tif')

    for filename in files:
        #filename = files[0]
        #print (files[0])
        out = Path(filename)
        #print(out.name)
        parent = out.resolve().parent
        #print(parent)

        label_file = glob.glob(str(parent) + '/*.json')[0]
        #print (label_file)
        #name = str(out.name).split("__")[1].split("--")


        with open(label_file) as json_file:
            data = json.load(json_file)
        name = data['labels']
        classes_arr.append([str(out), name])
    
    return pd.DataFrame(classes_arr, columns=['files', 'labels'])
    
    #for filename in files:
    #    out = Path(filename)
    #    name = str(out.name).split("__")[1].split("--")
    #    classes_arr.append([str(out), name])
    #return pd.DataFrame(classes_arr, columns=['files', 'labels'])

path_data = "data"
# load data paths to dictionaries.
dict_train = get_data("/root/tensorflow_datasets/train/")
dict_test = get_data("/root/tensorflow_datasets/test/")
dict_val = get_data("/root/tensorflow_datasets/val/")

In [30]:
mlb = MultiLabelBinarizer()
mlb.fit(dict_train['labels'])
print("Labels:")
for (i, label) in enumerate(mlb.classes_):
    print("{}. {}".format(i, label))

Labels:
0. Agro-forestry areas
1. Airports
2. Annual crops associated with permanent crops
3. Bare rock
4. Beaches, dunes, sands
5. Broad-leaved forest
6. Burnt areas
7. Coastal lagoons
8. Complex cultivation patterns
9. Coniferous forest
10. Construction sites
11. Continuous urban fabric
12. Discontinuous urban fabric
13. Dump sites
14. Estuaries
15. Fruit trees and berry plantations
16. Green urban areas
17. Industrial or commercial units
18. Inland marshes
19. Intertidal flats
20. Land principally occupied by agriculture, with significant areas of natural vegetation
21. Mineral extraction sites
22. Mixed forest
23. Moors and heathland
24. Natural grassland
25. Non-irrigated arable land
26. Olive groves
27. Pastures
28. Peatbogs
29. Permanently irrigated land
30. Port areas
31. Rice fields
32. Road and rail networks and associated land
33. Salines
34. Salt marshes
35. Sclerophyllous vegetation
36. Sea and ocean
37. Sparsely vegetated areas
38. Sport and leisure facilities
39. Transit

In [31]:
# Create Data Iterators
''' Data generator for training data'''
train_datagen = ImageDataGenerator(rescale=1. / 255)
train_generator = train_datagen.flow_from_dataframe(
    dict_train,
    x_col='files',
    y_col='labels',
    target_size=cfg.img_size_dest,
    batch_size=cfg.train_batchsize,
    class_mode='categorical',
)
### Now compute class weights from this

from sklearn.utils import class_weight
import sys
weight_sum = sum([cfg.weights[x] for x in cfg.weights])
print("Total Weight: %s" % weight_sum)
weights = {y:(cfg.weights[x] / weight_sum) for y,x in enumerate(mlb.classes_)}

Found 58008 validated image filenames belonging to 43 classes.
Total Weight: 781663


In [32]:
''' Data generator for test data'''
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_dataframe(
    dict_test,
    x_col='files',
    y_col='labels',
    target_size=cfg.img_size_dest,
    batch_size=cfg.test_batchsize,
    class_mode='categorical',
    shuffle=False
)

''' Data generator for validation data'''
validation_datagen = ImageDataGenerator(rescale=1. / 255)
validation_generator = validation_datagen.flow_from_dataframe(
    dict_val,
    x_col='files',
    y_col='labels',
    target_size=cfg.img_size_dest,
    batch_size=cfg.val_batchsize,
    class_mode='categorical',
    shuffle=False
)

Found 62748 validated image filenames belonging to 43 classes.
Found 41016 validated image filenames belonging to 43 classes.


In [33]:
# get number of cores
num_cpus = multiprocessing.cpu_count()
print("Number of cores: %s" %num_cpus)


# Create a callback for storing the model. This will be used for quantization later

checkpoint = tf.keras.callbacks.ModelCheckpoint("best-model.ckpt",
                              monitor="val_loss", mode="min",
                              #monitor="val_acc", mode="max",
                              save_best_only=True, verbose=1)

Number of cores: 32


In [34]:
# Load Model from Google
hub_layer = hub.KerasLayer("https://tfhub.dev/google/remote_sensing/bigearthnet-resnet50/1", trainable=False)

# Add Layers to predict our dataset
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(43, activation='sigmoid'))

In [35]:
# Compile new model
model.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.SGD(learning_rate=cfg.lr, momentum=0.9),
        metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()] 
)

In [36]:
history = model.fit(
        train_generator,
        epochs=cfg.num_epochs,
        validation_steps=validation_generator.samples/validation_generator.batch_size,
        steps_per_epoch=train_generator.samples/train_generator.batch_size,
        validation_data=validation_generator,
        callbacks=[checkpoint], #checkpoint = ModelCheckpoint(fname
        workers=num_cpus,
        #class_weight=weights,
        verbose=1
)

Epoch 1/10
580/580 [==============================] - 205s 349ms/step - loss: 0.1950 - precision_3: 0.4544 - recall_3: 0.1244 - val_loss: 0.1626 - val_precision_3: 0.6853 - val_recall_3: 0.1199

Epoch 00001: val_loss improved from inf to 0.16255, saving model to best-model.ckpt
INFO:tensorflow:Assets written to: best-model.ckpt/assets


INFO:tensorflow:Assets written to: best-model.ckpt/assets


Epoch 2/10
580/580 [==============================] - 211s 361ms/step - loss: 0.1568 - precision_3: 0.6778 - recall_3: 0.1509 - val_loss: 0.1611 - val_precision_3: 0.6388 - val_recall_3: 0.1501

Epoch 00002: val_loss improved from 0.16255 to 0.16112, saving model to best-model.ckpt
INFO:tensorflow:Assets written to: best-model.ckpt/assets


INFO:tensorflow:Assets written to: best-model.ckpt/assets


Epoch 3/10
581/580 [==============================] - ETA: 0s - loss: 0.1544 - precision_3: 0.6842 - recall_3: 0.1595

KeyboardInterrupt: 

In [37]:
from sklearn import metrics 
predictions = model.predict(test_generator, verbose=1)
ground_truth = test_generator.classes

y_true = MultiLabelBinarizer().fit_transform(ground_truth)
y_pred = np.rint(predictions)
y_score = predictions


f = metrics.f1_score(y_true, y_pred, average='samples')
p = metrics.precision_score(y_true, y_pred, average='samples')
r = metrics.recall_score(y_true, y_pred, average='samples')
AP = metrics.average_precision_score(y_true, y_score)
lrap = metrics.label_ranking_average_precision_score(y_true, y_score)
ji = metrics.jaccard_score(y_true, y_pred, average="samples")
#mAP = mAP(y_true, y_score)
hl = metrics.hamming_loss(y_true, y_pred)
rl = metrics.label_ranking_loss(y_true, y_score)
cov = metrics.coverage_error(y_true, y_score)

print("------------------------------")
print("F1: ", f)
print("presision: ",p)
print("Recall: ", r)
print("AP: ", AP) # edit to mAP
print("lrap: ", lrap)
print("Jaccard Index: ", ji)
#print("mAP", mAP)
print("Hamming Loss: ", hl)
print("Ranking Loss: ", rl)
print("Coverage Error: ", cov)

 29/628 [>.............................] - ETA: 1:29

KeyboardInterrupt: 